In [ ]:
import time
from pynput import keyboard, mouse
from threading import Thread
import tkinter as tk

# Global variables to track activity
last_key_press_time = time.time()
last_mouse_click_time = time.time()

def on_press(key):
    global last_key_press_time
    last_key_press_time = time.time()
    # You can add code here to log the time or do something else

def on_click(x, y, button, pressed):
    global last_mouse_click_time
    if pressed:
        last_mouse_click_time = time.time()
        # You can add code here to log the time or do something else

def ask_focus_level():
    while True:
        time.sleep(900)  # Wait for 15 minutes
        root = tk.Tk()
        root.title("Focus Level")
        
        def submit():
            focus_level = scale.get()  # Get the focus level from the slider
            root.destroy()
            # Here you can log the focus level or do something else with it
            
        tk.Label(root, text="Rate your focus level:").pack()
        scale = tk.Scale(root, from_=0, to=10, orient='horizontal')
        scale.pack()
        tk.Button(root, text="Submit", command=submit).pack()
        
        root.mainloop()

# Start monitoring keyboard and mouse
keyboard_listener = keyboard.Listener(on_press=on_press)
mouse_listener = mouse.Listener(on_click=on_click)

keyboard_listener.start()
mouse_listener.start()

# Start the focus level prompt in a separate thread
focus_thread = Thread(target=ask_focus_level)
focus_thread.start()

keyboard_listener.join()
mouse_listener.join()
focus_thread.join()
